# simulate stock / global etf return rate over next X years

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
# get data about LON:ISAC monthly from the last 5 years

# isac = yf.Ticker("VOO")
isac = yf.Ticker("ACWI")
isac_hist = isac.history(period="25y", interval="1mo")
isac_hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2008-04-01 00:00:00-04:00,36.996590,44.506346,36.930327,39.205341,242600,0.000,0.0,0.0
2008-05-01 00:00:00-04:00,38.763594,40.670483,38.358654,39.544018,1185200,0.000,0.0,0.0
2008-06-01 00:00:00-04:00,39.529280,39.588181,36.002643,36.385490,409000,0.000,0.0,0.0
2008-07-01 00:00:00-04:00,35.936381,37.327893,33.977955,35.236942,3255200,0.000,0.0,0.0
2008-08-01 00:00:00-04:00,35.192769,35.428369,33.801256,34.647945,500000,0.000,0.0,0.0
...,...,...,...,...,...,...,...,...
2023-12-01 00:00:00-05:00,96.908687,101.316776,96.740290,100.811577,82805700,0.962,0.0,0.0
2024-01-01 00:00:00-05:00,101.010002,103.610001,99.260002,102.050003,129985700,0.000,0.0,0.0
2024-02-01 00:00:00-05:00,102.389999,107.110001,102.239998,106.650002,82365600,0.000,0.0,0.0


In [4]:
returns = isac_hist['Close'].pct_change()
returns

Date
2008-04-01 00:00:00-04:00         NaN
2008-05-01 00:00:00-04:00    0.008639
2008-06-01 00:00:00-04:00   -0.079874
2008-07-01 00:00:00-04:00   -0.031566
2008-08-01 00:00:00-04:00   -0.016715
                               ...   
2023-12-01 00:00:00-05:00    0.038257
2024-01-01 00:00:00-05:00    0.012285
2024-02-01 00:00:00-05:00    0.045076
2024-03-01 00:00:00-05:00    0.032630
2024-04-01 00:00:00-04:00   -0.041133
Name: Close, Length: 193, dtype: float64

In [5]:
mu, sigma = returns.mean(), returns.std()
initial_price = isac_hist['Close'].iloc[-1]

print(f"Mean: {mu}")
print(f"Std: {sigma}")
# plot returns
fr = px.line(returns, title="Monthly returns")
fr.show()

Mean: 0.0064541256650307065
Std: 0.050380692500993744


In [6]:
years = 40
sim_rets = np.random.normal(mu, sigma, years*12)


In [7]:
sim_price = initial_price * (sim_rets + 1).cumprod()

px.line(x=np.arange(0, years, 1/12), y=sim_price, title="Simulated ISAC.L price over 40 years")

In [8]:
fig = px.line(x=np.arange(0, years, 1/12), y=sim_price, title="Simulated ISAC.L price over 40 years")
# accumulate 100 simulations

accumulated = np.zeros(years*12)
sim_all = []
sim_ret_all = []

for i in range(100):
    sim_rets = np.random.normal(mu, sigma, years*12)
    sim_ret_all.append(sim_rets)
    sim_price = initial_price * (sim_rets + 1).cumprod()
    sim_all.append(sim_price)
    fig.add_scatter(x=np.arange(0, years, 1/12), y=sim_price, mode='lines', line=dict(color='gray', width=0.5))

fig.show()

# get the mean from the accumulated results

In [9]:

the_mean = np.array(sim_all).mean(axis=0)
rets_mean = np.array(sim_ret_all).mean(axis=0)
# plot the mean
# fig.add_scatter(x=np.arange(0, years, 1/12), y=the_mean, mode='lines', line=dict(color='red', width=2))

f1 = px.line(x=np.arange(0, years, 1/12), y=rets_mean, title="Simulated")
f1.show()

In [10]:
# save to the file with date and change
beginning_of_month = pd.Timestamp.today().replace(day=1).date()    
time_from_now = pd.date_range(start=beginning_of_month, periods=years*12, freq='ME')
sim_df = pd.DataFrame(rets_mean)
sim_df.index = time_from_now
sim_df.rename_axis('date', inplace=True)
sim_df.rename(columns={0: 'value'}, inplace=True)
sim_df['value'] = round(sim_df['value'], 4)

In [11]:
# NBVAL_SKIP
sim_df.to_csv("../data/acwi_monthly_simulation.csv")